# Experiment 07: Baseline DeiT-Tiny

# Importing Essential Libraries

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torchvision.models as models
import timm
import time
from sklearn.metrics import classification_report, accuracy_score

# 1.0 Data Loading and Apply Augmentation

In [ ]:
def get_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),  # Normalization for CIFAR-10
    ])

# Loading CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=get_transforms())
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=get_transforms())
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

100%|██████████| 170498071/170498071 [00:05<00:00, 30782635.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 2.0 Model Instantiation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deit_model = timm.create_model('deit_tiny_patch16_224', pretrained=False, num_classes=10).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(deit_model.parameters(), lr=0.001)

# 3.0 Model Training

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    print('Finished Training')
    return model

# Call training function
trained_model = train_model(deit_model, criterion, optimizer)

Epoch 1/50, Loss: 2.0106
Epoch 2/50, Loss: 1.9665
Epoch 3/50, Loss: 1.9914
Epoch 4/50, Loss: 1.9329
Epoch 5/50, Loss: 1.8446
Epoch 6/50, Loss: 1.8299
Epoch 7/50, Loss: 1.8047
Epoch 8/50, Loss: 1.8711
Epoch 9/50, Loss: 1.7594
Epoch 10/50, Loss: 1.7159
Epoch 11/50, Loss: 1.6555
Epoch 12/50, Loss: 1.5979
Epoch 13/50, Loss: 1.5699
Epoch 14/50, Loss: 1.5347
Epoch 15/50, Loss: 1.5025
Epoch 16/50, Loss: 1.5052
Epoch 17/50, Loss: 1.4542
Epoch 18/50, Loss: 1.4552
Epoch 19/50, Loss: 1.4243
Epoch 20/50, Loss: 1.4227
Epoch 21/50, Loss: 1.4166
Epoch 22/50, Loss: 1.3897
Epoch 23/50, Loss: 1.3743
Epoch 24/50, Loss: 1.3826
Epoch 25/50, Loss: 1.4370
Epoch 26/50, Loss: 1.3861
Epoch 27/50, Loss: 1.3658
Epoch 28/50, Loss: 1.3536
Epoch 29/50, Loss: 1.3522
Epoch 30/50, Loss: 1.3337
Epoch 31/50, Loss: 1.4007
Epoch 32/50, Loss: 1.4477
Epoch 33/50, Loss: 1.3555
Epoch 34/50, Loss: 1.3183
Epoch 35/50, Loss: 1.3068
Epoch 36/50, Loss: 1.2870
Epoch 37/50, Loss: 1.2883
Epoch 38/50, Loss: 1.2950
Epoch 39/50, Loss: 1.

# 4.0 Model Evaluation

In [ ]:
# Generate classification report
all_preds = []
all_targets = []
deit_model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = deit_model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_targets, all_preds)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:\n', classification_report(all_targets, all_preds))

Accuracy: 50.51%
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.54      0.56      1000
           1       0.50      0.60      0.55      1000
           2       0.42      0.41      0.41      1000
           3       0.39      0.31      0.35      1000
           4       0.47      0.44      0.45      1000
           5       0.45      0.38      0.41      1000
           6       0.52      0.61      0.56      1000
           7       0.55      0.47      0.51      1000
           8       0.67      0.61      0.64      1000
           9       0.49      0.68      0.57      1000

    accuracy                           0.51     10000
   macro avg       0.50      0.51      0.50     10000
weighted avg       0.50      0.51      0.50     10000

